In [ ]:
# pseudo-code

# 무신사 사이트 접속

# 상단 검색바 클릭

# 인기검색어 > 1등 키워드 서칭

# 니트

# 니트 키워드 입력 > 니트 상품 PLP

# 니트 관련 상품 목록 > 인기 상품 5개 선정

# 인기 상품 5개 각각의 PDP 접속

# 각 상품별 리뷰(댓글) 목록 확인

# 해당 리뷰(댓글) 수집

# 엑셀 저장

# MySQL 저장

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

# 상수값으로서의 변수 목록
URL = "https://www.musinsa.com/main/musinsa/recommend?gf=A"

service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(2)

search_button = driver.find_element(By.CLASS_NAME, "Searchbar__TextFieldWrap-sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_element(By.CLASS_NAME, "search-home-popular-wrap")
items = popular_keywords.find_elements(By.TAG_NAME, "li")
target_keyword = items[0].text
time.sleep(2)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
search_input.send_keys(Keys.RETURN)
time.sleep(2)

print("타겟키워드 :", target_keyword)
print("페이지타이틀 :", driver.title)

driver.quit()

타겟키워드 : 니트
페이지타이틀 : 니트 | 무신사 추천 상품


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

# 상수값으로서의 변수 목록
URL = "https://www.musinsa.com/main/musinsa/recommend?gf=A"

service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(2)

search_button = driver.find_element(By.CLASS_NAME, "Searchbar__TextFieldWrap-sc-dzagxs-2")
search_button.click()
time.sleep(2)

popular_keywords = driver.find_element(By.CLASS_NAME, "search-home-popular-wrap")
items = popular_keywords.find_elements(By.TAG_NAME, "li")
target_keyword = items[0].text
time.sleep(2)

search_input = driver.find_element(By.CLASS_NAME, "search-home-search-bar-keyword")
search_input.clear()
search_input.send_keys(target_keyword)
search_input.send_keys(Keys.RETURN)
time.sleep(2)

product_data = list()
# product_data = []

for _ in range(2) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    products = driver.find_elements(By.CSS_SELECTOR, "div.sc-cpclqO.jrLXZA")[:5]

    # for product in products :
    #     try :
    #         product_info = product.find_element(By.CSS_SELECTOR, "div.sc-itBLYH.cDYUnS")
    #         brand = product_info.find_element(By.CSS_SELECTOR, "span.text-etc_11px_semibold").text.strip()
    #         name = product_info.find_element(By.CSS_SELECTOR, "span.text-body_13px_reg").text.strip()
    #         url = product_info.find_element(By.CSS_SELECTOR, "a.gtm-select-item").get_attribute("href")
    #         review_infos = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")
    #         for review_info in review_infos :
    #             review_rating = review_info[0]
    #             review_count = review_info[1]
                
    #             product_data.append({"브랜드명" : brand, "상품명": name, "URL": url, "평점": review_rating, "리뷰수": review_count})
    #     except :
    #         continue
            
    for product in products :
        product_info = product.find_element(By.CSS_SELECTOR, "div.sc-itBLYH.cDYUnS")
        brand = product_info.find_element(By.CSS_SELECTOR, "span.text-etc_11px_semibold").text.strip()
        name = product_info.find_element(By.CSS_SELECTOR, "span.text-body_13px_reg").text.strip()
        url = product_info.find_element(By.CSS_SELECTOR, "a.gtm-select-item").get_attribute("href")
        # review_infos = product.find_elements(By.CSS_SELECTOR, "span.text-etc_11px_reg.text-yellow.font-pretendard")
        # print(review_infos)
        # for review_info in review_infos :
        #     review_rating = review_info[0]
        #     review_count = review_info[1]
            
        product_data.append({"브랜드명" : brand, "상품명": name, "URL": url})

print(product_data)
driver.quit()

[{'브랜드명': '수아레', '상품명': '워셔블 하찌 니트 - 12 COLOR', 'URL': 'https://www.musinsa.com/products/2396645'}, {'브랜드명': '수아레', '상품명': '워셔블 케이블 라운드 니트 - 5 COLOR', 'URL': 'https://www.musinsa.com/products/3494992'}, {'브랜드명': '피지컬 디파트먼트', '상품명': '하트애플 크롭 니트_차콜', 'URL': 'https://www.musinsa.com/products/4344303'}, {'브랜드명': '커버낫', '상품명': '램스울 카라 니트 네이비', 'URL': 'https://www.musinsa.com/products/3487580'}, {'브랜드명': '키뮤어', '상품명': '[유튜버 PICK] 워셔블 카라니트 풀집업_5 COLOR', 'URL': 'https://www.musinsa.com/products/4348898'}, {'브랜드명': '비버리힐스 폴로클럽', '상품명': '여성 하프 집업 스웻 셔츠 3종택1 TS006', 'URL': 'https://www.musinsa.com/products/4582950'}, {'브랜드명': '라운드트리코', '상품명': '홀터 라운드넥 니트 - 블랙', 'URL': 'https://www.musinsa.com/products/3546644'}, {'브랜드명': '레터프롬문', '상품명': '원 숄더 레이어드 니트 탑 ( 멜란지그레이 )', 'URL': 'https://www.musinsa.com/products/4555007'}, {'브랜드명': '브이투', '상품명': '[SET]데일리 워셔블 소프트 하찌 니트 + 케이블 소프트 니트_8COLOR', 'URL': 'https://www.musinsa.com/products/4331963'}, {'브랜드명': '커즈넬로', '상품명': '멀티 블랜드 브이넥 니트_브라운', 'URL': 'https://w